In [22]:
import pandas as pd

In [23]:
def clean_df(df, extra_col):
    # Drop first row (empty)
    df = df.drop(df.index[0])

    # Add year column name
    df.iloc[0, 0] = "year"

    # Make years the header
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])

    # Drop all rows starting with Nan
    df = df.dropna(subset=[df.columns[0]])

    # # Remove indicator group titles
    patterns_to_remove = [
        "Utemiljö",
        "Missbruksproblem", 
        "Utomhusstörningar",
        "Andel uppfattat minst ett problem",
        "Utsatthet för brott",
        "Oro för att utsättas för brott",
        "Konkret känsla av otrygghet",
        "Polisens agerande mot problem",
        "Tillit"
    ]
    mask = ~df.iloc[:, 0].astype(str).str.startswith(tuple(patterns_to_remove))
    df = df[mask]

    # Transpose df
    df = df.transpose()

    # Make the first row the header
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])

    # Reset index
    df = df.reset_index()

    # Rename first column to "year"
    df.columns.values[0] = "year"

    # Remove rows with 'Year' == '2020_1' or '2016_1'
    df = df[~df['year'].isin(['2020_1', '2016_1'])]

    # Rename '2020_2' to '2020' and '2016_2' to '2016', etc
    df['year'] = df['year'].replace({'2020_2': '2020', '2016_2': '2016', '2006*': '2006'})
    
    # Rename special characters ä, å and ö
    df.columns = df.columns.str.replace('ä', 'a').str.replace('å', 'a').str.replace('ö', 'o')

    # Rename columns to snake_casing for easier coding
    df.columns = df.columns.str.replace(' ', '_').str.replace('.', '').str.replace(',', '').str.lower()

    # Transform years to numbers
    df["year"] = df["year"].astype(int)

    # Transform numeric columns to numbers
    df = df.apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',', '.'), errors="coerce") if x.dtype == 'object' else x)

    # Add extra column
    if isinstance(extra_col, dict):
        for col_name, col_value in extra_col.items():
            df.insert(0, col_name, col_value)

    return df

In [24]:
annat_raw = pd.read_excel('Resultatbild - Annat.xlsx')
kvinna_raw = pd.read_excel('Resultatbild - Kvinna.xlsx')
man_raw = pd.read_excel('Resultatbild - Man.xlsx')

# Clean all raw data
annat_df = clean_df(annat_raw, {'kon': 'Annat'})
kvinna_df = clean_df(kvinna_raw, {'kon': 'Kvinna'})
man_df = clean_df(man_raw, {'kon': 'Man'})

# Join clean dfs
df = pd.concat([
    annat_df,
    kvinna_df,
    man_df
])

print(f'{len(df)} entries')
df.sample(10)

50 entries


year,kon,year,nedskrapning,skadegorelse,berusade_personer_utomhus,narkotikapaverkade_personer_utomhus,bostader_tillhall_for_alkoholmissbrukare,bostader_tillhall_for_narkotikamissbrukare,observerat_narkotikaforsaljning_i_omradet,folk_brakar_och_slass_utomhus,...,restaurang_bar_eller_disco,sportevenemang,foreningsmoten_kurser_och_liknande,åka_buss_eller_tag,andel_avstatt_fran_nagon_typ_av_aktivitet,polisen_bryr_sig_om_de_lokala_problemen,polisen_bryr_sig_inte_om_de_lokala_problemen,relationskvot,boende_skulle_ej_agera_vid_slagsmal,svag_sammanhallning_i_bostadsomradet
8,Annat,2017,42.64,32.82,41.40,34.95,21.04,21.03,NaN,23.13,...,5.45,5.65,9.35,13.98,22.43,34.37,26.06,75.83,28.12,35.57
19,Kvinna,2007,36.36,38.58,18.00,11.80,8.56,7.87,NaN,13.77,...,6.31,2.10,3.29,9.23,13.37,37.54,11.64,31.00,NaN,NaN
18,Man,2008,32.88,37.36,16.17,10.53,8.08,7.59,NaN,11.87,...,5.28,1.62,1.06,4.07,8.49,41.19,16.20,39.34,NaN,NaN
2,Man,2022,36.11,32.94,14.32,14.36,7.33,8.44,10.76,11.82,...,7.15,3.88,2.38,9.00,13.02,50.79,13.50,26.58,19.20,11.89
14,Man,2012,30.34,27.97,13.86,9.44,6.65,6.63,NaN,8.85,...,4.40,2.42,1.43,4.66,8.47,45.08,14.42,31.98,NaN,NaN
9,Annat,2016,40.62,38.61,23.82,18.24,22.63,19.29,NaN,23.27,...,11.84,11.44,7.38,14.42,22.42,22.40,31.47,140.48,NaN,NaN
21,Man,2005,24.65,28.34,14.57,7.92,6.65,5.58,NaN,8.04,...,4.21,1.07,0.78,2.04,6.01,40.37,16.19,40.11,NaN,NaN
19,Man,2007,34.81,41.03,18.58,11.88,8.64,8.33,NaN,12.61,...,5.82,2.43,1.28,5.19,10.66,43.10,15.97,37.06,NaN,NaN
11,Man,2015,26.74,25.11,12.52,9.74,6.75,6.51,NaN,8.06,...,4.26,3.93,1.57,5.29,9.52,46.55,13.57,29.15,NaN,NaN
13,Kvinna,2013,33.34,29.13,15.37,10.34,6.05,6.04,NaN,9.15,...,4.61,2.14,2.32,8.08,11.76,40.15,11.62,28.93,NaN,NaN


In [25]:
df.to_csv('kon.csv')